## Import libraries

In [90]:
import pandas as pd
import os
from datetime import datetime


## Set Working Directory

In [91]:
# Define the new working directory path
new_working_directory = r'C:\cloudresume\react\resume\sports-data'

# Change the current working directory
os.chdir(new_working_directory)

## Import Datasets

In [92]:
# Define the path 
data_folder = 'data/'

# Load the CSV files to df
appearances_df = pd.read_csv(data_folder + 'appearances_filtered.csv')
players_df = pd.read_csv(data_folder + 'players_filtered.csv')

In [93]:
appearances_df.head()

,Unnamed: 0,player_id,yellow_cards,red_cards,goals,assists,minutes_played,game_id,date
0,4,122011,0,0,0,1,90,2234421,2012-07-05
1,24,14940,0,0,0,0,90,2234404,2012-07-09
2,25,14942,0,0,0,1,90,2234404,2012-07-09
3,29,26267,0,0,0,1,90,2234404,2012-07-09
4,39,55735,0,0,0,0,90,2234404,2012-07-09


In [94]:
players_df.head()

,Unnamed: 0,name,current_club_id,country_of_citizenship,date_of_birth,position,foot,height_in_cm,market_value_in_eur,highest_market_value_in_eur,player_id,current_club_domestic_competition_id
0,0,Miroslav Klose,398,Germany,1978-06-09,Attack,right,184.0,1000000.0,30000000.0,10,IT1
1,1,Roman Weidenfeller,16,Germany,1980-08-06,Goalkeeper,left,190.0,750000.0,8000000.0,26,L1
2,3,Lúcio,506,Brazil,1978-05-08,Defender,NaN,NaN,200000.0,24500000.0,77,IT1
3,4,Tom Starke,27,Germany,1981-03-18,Goalkeeper,right,194.0,100000.0,3000000.0,80,L1
4,6,Christoph Metzelder,33,Germany,1980-11-05,Defender,NaN,NaN,1500000.0,9500000.0,123,L1


# Merge both Dataframes and calculate players stats

In [95]:
# Merge the dfs
merged_df = appearances_df.merge(players_df, on='player_id')

# Calculate the number of games played per player
number_games_played = merged_df.groupby('player_id')['game_id'].nunique().reset_index(name='number_games_played')

# Calculate the total minutes played per player
total_minutes = merged_df.groupby('player_id')['minutes_played'].sum().reset_index(name='total_minutes')

# Calculate the average minutes played per player
average_minutes = merged_df.groupby('player_id')['minutes_played'].mean().reset_index(name='average_minutes')

# Calculate the sum of yellow cards per player
yellow_cards_sum = merged_df.groupby('player_id')['yellow_cards'].sum().reset_index(name='yellow_cards_sum')

# Calculate the average number of yellow cards per game per player
yellow_cards_avg = merged_df.groupby('player_id')['yellow_cards'].mean().reset_index(name='yellow_cards_avg')

# Calculate the sum of red cards per player
red_cards_sum = merged_df.groupby('player_id')['red_cards'].sum().reset_index(name='red_cards_sum')

# Calculate the average number of red cards per game per player
red_cards_avg = merged_df.groupby('player_id')['red_cards'].mean().reset_index(name='red_cards_avg')

# Calculate the total goals per player
goals = merged_df.groupby('player_id')['goals'].sum().reset_index(name='goals')

# Calculate the average goals per game per player
avg_goals_per_game = merged_df.groupby('player_id')['goals'].mean().reset_index(name='avg_goals_per_game')

# Calculate the total assists per player
assists = merged_df.groupby('player_id')['assists'].sum().reset_index(name='assists')

# Extract the desired columns from the merged dataframe
desired_columns = ['player_id', 'name', 'country_of_citizenship', 'height_in_cm', 'foot', 'position', 'highest_market_value_in_eur', 'current_club_domestic_competition_id']
desired_df = merged_df[desired_columns].drop_duplicates()

# Merge all the calculated statistics dataframes with the desired dataframe
final_df = desired_df.merge(number_games_played, on='player_id')
final_df = final_df.merge(total_minutes, on='player_id')
final_df = final_df.merge(average_minutes, on='player_id')
final_df = final_df.merge(yellow_cards_sum, on='player_id')
final_df = final_df.merge(yellow_cards_avg, on='player_id')
final_df = final_df.merge(red_cards_sum, on='player_id')
final_df = final_df.merge(red_cards_avg, on='player_id')
final_df = final_df.merge(goals, on='player_id')
final_df = final_df.merge(avg_goals_per_game, on='player_id')
final_df = final_df.merge(assists, on='player_id')

# Rename the columns for clarity
final_df = final_df.rename(columns={
    'country_of_citizenship': 'country',
    'height_in_cm': 'height',
    'highest_market_value_in_eur': 'highest_market_value'
})



## calculate player age as of today

In [96]:
# Convert the 'date_of_birth' column to datetime
players_df['date_of_birth'] = pd.to_datetime(players_df['date_of_birth'])

In [97]:
from datetime import date

# Define the function to calculate age
def calculateAge(birthDate):
    today = date.today()
    age = today.year - birthDate.year - ((today.month, today.day) < (birthDate.month, birthDate.day))
    return age

# Convert 'date_of_birth' column to datetime
players_df['date_of_birth'] = pd.to_datetime(players_df['date_of_birth'])

# Apply the calculateAge function to each row in the DataFrame
final_df['age'] = players_df['date_of_birth'].apply(lambda x: calculateAge(x.date()))


In [98]:
final_df.head()

,player_id,name,country,height,foot,position,highest_market_value,current_club_domestic_competition_id,number_games_played,total_minutes,average_minutes,yellow_cards_sum,yellow_cards_avg,red_cards_sum,red_cards_avg,goals,avg_goals_per_game,assists,age
0,122011,Markus Henriksen,Norway,187.0,right,Defender,5000000.0,GB1,165,12199,73.933333,15,0.090909,1,0.006061,33,0.200000,22,45.0
1,14940,Razvan Rat,Romania,179.0,left,Defender,6500000.0,ES1,97,7690,79.278351,18,0.185567,1,0.010309,3,0.030928,13,43.0
2,14942,Darijo Srna,Croatia,182.0,right,Defender,17500000.0,IT1,227,19598,86.334802,59,0.259912,2,0.008811,22,0.096916,68,45.0
3,26267,Fernandinho,Brazil,179.0,right,Midfield,32000000.0,GB1,399,30325,76.002506,100,0.250627,3,0.007519,29,0.072682,41,42.0
4,55735,Henrikh Mkhitaryan,Armenia,177.0,both,Midfield,37000000.0,IT1,485,35878,73.975258,59,0.121649,0,0.000000,128,0.263918,119,43.0


In [99]:
appearances_df

,Unnamed: 0,player_id,yellow_cards,red_cards,goals,assists,minutes_played,game_id,date
0,4,122011,0,0,0,1,90,2234421,2012-07-05
1,24,14940,0,0,0,0,90,2234404,2012-07-09
2,25,14942,0,0,0,1,90,2234404,2012-07-09
3,29,26267,0,0,0,1,90,2234404,2012-07-09
4,39,55735,0,0,0,0,90,2234404,2012-07-09
...,...,...,...,...,...,...,...,...,...
661850,1530225,687609,1,0,0,1,76,4113038,2024-02-05
661851,1530226,730581,0,0,0,0,90,4113038,2024-02-05
661852,1530227,73517,0,0,0,0,90,4113038,2024-02-05
661853,1530228,82503,0,0,0,0,90,4113038,2024-02-05


# calculate stats per year

In [100]:
appearances_df['date'] = pd.to_datetime(appearances_df['date'])
appearances_df['year'] = appearances_df['date'].dt.year

In [101]:
appearances_df.head()

,Unnamed: 0,player_id,yellow_cards,red_cards,goals,assists,minutes_played,game_id,date,year
0,4,122011,0,0,0,1,90,2234421,2012-07-05,2012
1,24,14940,0,0,0,0,90,2234404,2012-07-09,2012
2,25,14942,0,0,0,1,90,2234404,2012-07-09,2012
3,29,26267,0,0,0,1,90,2234404,2012-07-09,2012
4,39,55735,0,0,0,0,90,2234404,2012-07-09,2012


In [102]:
# Group by player_id and year to calculate sums
yearly_stats_df = appearances_df.groupby(['player_id', 'year']).agg(
    total_games=('game_id', 'count'),
    total_goals=('goals', 'sum'),
    total_assists=('assists', 'sum')
).reset_index()

In [103]:
yearly_stats_df

,player_id,year,total_games,total_goals,total_assists
0,10,2012,20,11,1
1,10,2013,29,9,4
2,10,2014,31,8,6
3,10,2015,36,12,8
4,10,2016,20,8,6
...,...,...,...,...,...
36193,1159022,2023,2,0,0
36194,1159022,2024,3,0,0
36195,1176345,2023,5,0,0
36196,1178474,2023,5,0,0


In [104]:
# Calculate the number of years per player
years_played_df = yearly_stats_df.groupby('player_id')['year'].nunique()



In [105]:
years_played_df

player_id
10         5
26         7
77         1
80         4
123        2
          ..
1111912    2
1141628    1
1159022    2
1176345    1
1178474    2
Name: year, Length: 8076, dtype: int64

In [106]:
final_df = pd.merge(final_df, years_played_df, on='player_id')

In [107]:
final_df.head()

,player_id,name,country,height,foot,position,highest_market_value,current_club_domestic_competition_id,number_games_played,total_minutes,average_minutes,yellow_cards_sum,yellow_cards_avg,red_cards_sum,red_cards_avg,goals,avg_goals_per_game,assists,age,year
0,122011,Markus Henriksen,Norway,187.0,right,Defender,5000000.0,GB1,165,12199,73.933333,15,0.090909,1,0.006061,33,0.200000,22,45.0,6
1,14940,Razvan Rat,Romania,179.0,left,Defender,6500000.0,ES1,97,7690,79.278351,18,0.185567,1,0.010309,3,0.030928,13,43.0,5
2,14942,Darijo Srna,Croatia,182.0,right,Defender,17500000.0,IT1,227,19598,86.334802,59,0.259912,2,0.008811,22,0.096916,68,45.0,8
3,26267,Fernandinho,Brazil,179.0,right,Midfield,32000000.0,GB1,399,30325,76.002506,100,0.250627,3,0.007519,29,0.072682,41,42.0,11
4,55735,Henrikh Mkhitaryan,Armenia,177.0,both,Midfield,37000000.0,IT1,485,35878,73.975258,59,0.121649,0,0.000000,128,0.263918,119,43.0,13


In [108]:
# Assuming `df` is your main DataFrame and it already includes a 'years_played' column
final_df['avg_games_per_year'] = final_df['number_games_played'] / final_df['year']
final_df['avg_goals_per_year'] = final_df['goals'] / final_df['year']
final_df['avg_assists_per_year'] = final_df['assists'] / final_df['year']

In [109]:
print(final_df.columns.tolist())

['player_id', 'name', 'country', 'height', 'foot', 'position', 'highest_market_value', 'current_club_domestic_competition_id', 'number_games_played', 'total_minutes', 'average_minutes', 'yellow_cards_sum', 'yellow_cards_avg', 'red_cards_sum', 'red_cards_avg', 'goals', 'avg_goals_per_game', 'assists', 'age', 'year', 'avg_games_per_year', 'avg_goals_per_year', 'avg_assists_per_year']


In [110]:
final_df.to_csv(data_folder + 'cleaned_df.csv', index=False)

In [111]:
# Define the path 
data_folder = 'data/'

# Load the CSV files to df
cleaned_df = pd.read_csv(data_folder + 'cleaned_df.csv')

In [115]:
cleaned_df['current_club_domestic_competition_id'].unique()

array(['GB1', 'ES1', 'IT1', 'L1', 'FR1'], dtype=object)